In [2]:
# SAMMY DO NOT RUN, this is just for my venv
import sys
sys.path.append('/Users/tunadorable/local-repos/next-concept-predictor/venv/lib/python3.11/site-packages')

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [4]:
# setting regular hyperparameters
d=4
v=12
l = 2
h=4

activation = nn.ReLU()
norm = nn.LayerNorm(d)

# our tokens being mapped to indices. obvi a stupidly minimally small vocab
vocabulary_dict = {" I":0,
          " think":1,
         " there":2,
         "fore":3,
         " am":4,
         "Every":5,
         " cloud":6,
         " has":7,
         " a":8,
         " silver":9,
         " lining":10,
         "<endoftext>":11}

single_seq_list = [[' I', ' think', ' there', 'fore', ' I', ' am', '<endoftext>']]
double_seq_list = [[' I', ' think', ' there', 'fore', ' I', ' am', '<endoftext>'],
               ['Every', ' cloud', ' has', ' a', ' silver', ' lining', '<endoftext>']]

# ofc
seed = 42069

# table of contents

1. regular next-token predictor functions
2. regular NTP 2-step inference walkthrough
3. regular NTP training loop
4. my next-concept predictor functions
5. NCP 3-step inference walkthrough
6. NCP training loop

### 1. regular next-token predictor functions

In [5]:
def initialize(seq_list, vocabulary_dict, verbose=False):
    global seed, i, n, d, v, l, h, activation, norm
    
    # embedding matrix
    #torch.manual_seed(seed)
    E = norm(torch.randn(v,d))
    if verbose:
        print("E: ", E.shape, E)
        
    # turn list of tokens into indices
    S_n = torch.tensor([[vocabulary_dict[word] for word in sentence] for sentence in seq_list])
    if verbose:
        print("S_n: ", S_n.dtype, S_n.shape, S_n)

    # starting off with the first token for each sequence
    if i==0: 
        S_i = S_n[:,i].unsqueeze(dim=1) 
    else: 
        S_i = S_n[:,0:i]
    if verbose:
        print(f"i={i}")
        print("S_i: ", S_i.dtype, S_i.shape, S_i)
        
    return E, S_n, S_i

def ffn(X_0, verbose=False):
    """
    simple feedforward layer. 
    only included this so that there'd actually be some variation in inference output
    """
    global seed, i, n, d, v, l, h, activation, norm
    
    # first linear layer
    #torch.manual_seed(seed)
    W_1 = torch.randn(d,d**2)
    if verbose:
        print("W_1: ", W_1.shape, W_1)
    X_1 = torch.matmul(X_0,W_1)
    if verbose:
        print("X_1: ", X_1.shape, X_1)

    # activation function
    X_2 = activation(X_1)
    if verbose:
        print("X_2: ", X_2.shape, X_2)

    # second linear layer
    #torch.manual_seed(seed)
    W_2 = torch.randn(d**2,d)
    if verbose:
        print("W_2: ", W_2.shape, W_2)
    X_3 = torch.matmul(X_2,W_2)
    if verbose:
        print("X_3: ", X_3.shape, X_3)
    
    return X_3

def layers(X_0):
    global seed, i, n, d, v, l, h, activation, norm
    
    for j in range(l):
        
        # imma just ignore MHA cuz i'm not messing with it
        # keeping FFN tho cuz i think i need nolinearity to ensure outputs differ from input
        
        if j == 0: 
            # first layer
            X_i_ = ffn(X_0)
            X_i = norm(X_0+X_i_)
        elif j == l-1: 
            # last layer
            X_i_ = ffn(X_i)
            X_f = norm(X_i+X_i_)
        else: 
            # all layers in the middle
            X_i_ = ffn(X_i)
            X_i = norm(X_i+X_i_)
        
    return X_f

# a function to give us a pretty read off of the sequences
def tensor_to_string(tensor, token_to_index):
    """
    Convert a PyTorch tensor of indices into readable strings using a mapping dictionary.

    :param tensor: PyTorch tensor of indices. Can be 1D or 2D.
    :param token_to_index: Dictionary mapping tokens to indices.
    :return: A single string if the tensor is 1D, or a list of strings if the tensor is 2D.
    """
    
    index_to_token = {v: k for k, v in token_to_index.items()}
    
    # Check if the tensor is 1D or 2D
    if len(tensor.shape) == 1:
        # Convert the 1D tensor to a single string
        return ' '.join([index_to_token[int(idx)] for idx in tensor])
    elif len(tensor.shape) == 2:
        # Convert the tensor to a list of lists and then to strings
        sequences = tensor.tolist()
        return [' '.join([index_to_token[int(idx)] for idx in seq]) for seq in sequences]
    else:
        raise ValueError("Tensor must be either 1D or 2D")

### 2. regular NTP 2-step inference walkthrough

In [29]:
### FIRST PASS
print("FIRST PASS")
verbose=False
single_seq = [[' I']]

i=1
print(f"i={i}")

E, _, S_i = initialize(single_seq, vocabulary_dict)
print("S_i: ", S_i.shape)
print("E: ", E.shape)
if verbose:
    print(S_i)
    print(E)

X_0 = F.embedding(S_i, E)
print("X_0: ", X_0.shape)
if verbose: print(X_0)

X_f = layers(X_0)
print("X_f: ", X_f.shape)
if verbose: print(X_f)

Y = torch.matmul(X_f,E.T)
print("Y: ", Y.shape)
if verbose: print(Y)

Z = Y[:,-1,:]
print("Z: ", Z.shape)
if verbose: print(Z)

P = F.softmax(Z, dim=-1)
print("P: ", P.shape, torch.sum(P,dim=1))
if verbose: print(P)

s_iplus1 = torch.max(P, dim=1).indices
print("s_iplus1: ", s_iplus1.shape)
if verbose: print(s_iplus1)

S_iplus1 = torch.cat((S_i, s_iplus1.unsqueeze(dim=1)), dim=1)
print("S_iplus1: ", S_iplus1.shape, tensor_to_string(S_iplus1, vocabulary_dict))
if verbose: print(S_iplus1)
    
### second pass
print("SECOND PASS")
i=2
print(f"i={i}")

S_i = S_iplus1

X_0 = F.embedding(S_i, E)
print("X_0: ", X_0.shape)
if verbose: print(X_0)

X_f = layers(X_0)
print("X_f: ", X_f.shape)
if verbose: print(X_f)

Y = torch.matmul(X_f,E.T)
print("Y: ", Y.shape)
if verbose: print(Y)

Z = Y[:,-1,:]
print("Z: ", Z.shape)
if verbose: print(Z)

P = F.softmax(Z, dim=-1)
print("P: ", P.shape, torch.sum(P,dim=1))
if verbose: print(P)

s_iplus1 = torch.max(P, dim=1).indices
print("s_iplus1: ", s_iplus1.shape)
if verbose: print(s_iplus1)

S_iplus1 = torch.cat((S_i, s_iplus1.unsqueeze(dim=1)), dim=1)
print("S_iplus1: ", S_iplus1.shape, tensor_to_string(S_iplus1, vocabulary_dict))
if verbose: print(S_iplus1)
    
# really i should loop it & add logic to make the whole thing stop when <endoftext> is predicted

FIRST PASS
i=1
S_i:  torch.Size([1, 1])
E:  torch.Size([12, 4])
X_0:  torch.Size([1, 1, 4])
X_f:  torch.Size([1, 1, 4])
Y:  torch.Size([1, 1, 12])
Z:  torch.Size([1, 12])
P:  torch.Size([1, 12]) tensor([1.0000], grad_fn=<SumBackward1>)
s_iplus1:  torch.Size([1])
S_iplus1:  torch.Size([1, 2]) [' I  there']
SECOND PASS
i=2
X_0:  torch.Size([1, 2, 4])
X_f:  torch.Size([1, 2, 4])
Y:  torch.Size([1, 2, 12])
Z:  torch.Size([1, 12])
P:  torch.Size([1, 12]) tensor([1.0000], grad_fn=<SumBackward1>)
s_iplus1:  torch.Size([1])
S_iplus1:  torch.Size([1, 3]) [' I  there  think']


### 3. regular NTP training loop

In [26]:
verbose = False

double_seq_list = [[' I', ' think', ' there', 'fore', ' I', ' am', '<endoftext>'],
               ['Every', ' cloud', ' has', ' a', ' silver', ' lining', '<endoftext>']]

for i in range(1,n):
    print(f"i={i}")
    
    # initialize
    if i == 1:
        E, S_n, _ = initialize(double_seq_list, vocabulary_dict)
        print("S_n: ", S_n.shape, S_n)
        print("E: ", E.shape)
        if verbose:
            print(S_n)
            print(E)
    
    # iterate S_i
    S_i = S_n[:,0:i]
    print("S_i: ", S_i.shape)
    if verbose:
        print(S_i)
        
    X_0 = F.embedding(S_i, E)
    print("X_0: ", X_0.shape)
    if verbose: print(X_0)

    X_f = layers(X_0)
    print("X_f: ", X_f.shape)
    if verbose: print(X_f)

    Y = torch.matmul(X_f,E.T)
    print("Y: ", Y.shape)
    if verbose: print(Y)

    Z = Y[:,-1,:]
    print("Z: ", Z.shape)
    if verbose: print(Z)
        
    # correct indices to be trained on
    s_iplus1 = S_n[:,i]
    print("s_iplus1: ", s_iplus1.shape, s_iplus1)
    
    # cross-entropy loss
    loss = F.cross_entropy(Z, s_iplus1)
    print("loss: ", loss.shape, loss)

i=1
S_n:  torch.Size([2, 7]) tensor([[ 0,  1,  2,  3,  0,  4, 11],
        [ 5,  6,  7,  8,  9, 10, 11]])
E:  torch.Size([12, 4])
S_i:  torch.Size([2, 1])
X_0:  torch.Size([2, 1, 4])
X_f:  torch.Size([2, 1, 4])
Y:  torch.Size([2, 1, 12])
Z:  torch.Size([2, 12])
s_iplus1:  torch.Size([2]) tensor([1, 6])
loss:  torch.Size([]) tensor(7.6477, grad_fn=<NllLossBackward0>)
i=2
S_i:  torch.Size([2, 2])
X_0:  torch.Size([2, 2, 4])
X_f:  torch.Size([2, 2, 4])
Y:  torch.Size([2, 2, 12])
Z:  torch.Size([2, 12])
s_iplus1:  torch.Size([2]) tensor([2, 7])
loss:  torch.Size([]) tensor(6.5355, grad_fn=<NllLossBackward0>)
i=3
S_i:  torch.Size([2, 3])
X_0:  torch.Size([2, 3, 4])
X_f:  torch.Size([2, 3, 4])
Y:  torch.Size([2, 3, 12])
Z:  torch.Size([2, 12])
s_iplus1:  torch.Size([2]) tensor([3, 8])
loss:  torch.Size([]) tensor(2.6618, grad_fn=<NllLossBackward0>)
i=4
S_i:  torch.Size([2, 4])
X_0:  torch.Size([2, 4, 4])
X_f:  torch.Size([2, 4, 4])
Y:  torch.Size([2, 4, 12])
Z:  torch.Size([2, 12])
s_iplus1:

### 4. my next-concept predictor functions

we'll be re-using initialize(), layers(), and tensor_to_string() from earlier

In [23]:
def initialize_conditionals(P_max, verbose = False):
    global seed, i, n, d, v, l, h, activation, norm, gamma_0
    
    Gamma_i = torch.ones(P_max.shape[0])*gamma_0
    
    A_i_bar = torch.zeros(P_max.shape[0])
    A_i_bar[P_max <= gamma_0] = 1
    if verbose: print("A_i_bar: ", A_i_bar.dtype, A_i_bar.shape, A_i_bar)
    
    not_A_i_bar = torch.zeros(P_max.shape[0])
    not_A_i_bar[P_max > gamma_0] = 1
    if verbose: print("not_A_i_bar: ", not_A_i_bar.dtype, not_A_i_bar.shape, not_A_i_bar)
    
    return Gamma_i, A_i_bar, not_A_i_bar

def iter_conditionals(P_max, A_iminus1_bar, Gamma_iminus1, t_max, gamma_0, verbose = False):
    
    Delta_gamma = (1-gamma_0)/t_max
    
    D_i = A_iminus1_bar*Delta_gamma
    if verbose: print("D_i: ", D_i.shape, D_i)
    
    Gamma_i = Gamma_iminus1 - D_i
    if verbose: print("Gamma_i: ", Gamma_i.shape, Gamma_i)
    
    A_i_bar = torch.zeros(P_max.shape[0])
    A_i_bar[P_max <= Gamma_i] = 1
    if verbose: print("A_i_bar: ", A_i_bar.dtype, A_i_bar.shape, A_i_bar)
    
    not_A_i_bar = torch.zeros(P_max.shape[0])
    not_A_i_bar[P_max > Gamma_i] = 1
    if verbose: print("not_A_i_bar: ", not_A_i_bar.dtype, not_A_i_bar.shape, not_A_i_bar)
    
    return Gamma_i, A_i_bar, not_A_i_bar

def build_concepts_tensor(Y, A_i_bar, C_i = None, verbose=False):
    global seed, i, n, d, v, l, h, activation, norm, gamma_0
    
    if verbose:
        print("Y: ", Y.shape)
        print("A_i_bar: ", A_i_bar.shape, A_i_bar)
    C_i_bar = Y*A_i_bar.unsqueeze(dim=1)
    
    if torch.is_tensor(C_i):
        prepend = C_i
    else:
        prepend = torch.zeros((Y.shape[0],i,d))
    
    if verbose:
        print("prepend: ", prepend.shape)
        print("C_i_bar: ", C_i_bar.shape, C_i_bar)
    C_iplus1 = torch.cat((prepend, C_i_bar.unsqueeze(dim=1)), dim=1)
    if verbose: print("C_iplus1: ", C_iplus1.shape, C_iplus1)
    
    return C_iplus1

def inference(A_i_bar, not_A_i_bar, s_iplus1, S_i, verbose=False):
    global seed, i, n, d, v, l, h, activation, norm, gamma_0
    
    if verbose:
        print("A_i_bar: ", A_i_bar.shape, A_i_bar)
        print("not_A_i_bar: ", not_A_i_bar.shape, not_A_i_bar, not_A_i_bar.type(torch.bool))
        print("s_iplus1: ", s_iplus1.shape, s_iplus1)
        
    A_i_bar_v = A_i_bar*v
    if verbose: print("A_i_bar_v: ", A_i_bar_v.shape, A_i_bar_v)
    
    s_iplus1_bar = s_iplus1*not_A_i_bar
    if verbose: print("s_iplus1_bar: ", s_iplus1_bar.shape, s_iplus1_bar)
    
    s_iplus1_bar_v = s_iplus1_bar + A_i_bar_v
    if verbose: print("s_iplus1_bar_v: ", s_iplus1_bar_v.shape, s_iplus1_bar_v)
    
    S_iplus1 = torch.cat((S_i, s_iplus1_bar_v.unsqueeze(dim=1)), dim=1)
    
    return S_iplus1

def custom_loss(logits, target, mask, verbose=False):
    
    # Apply LogSoftmax
    log_probs = F.log_softmax(logits, dim=1)
    if verbose: print("log_probs: ", log_probs.shape, log_probs)
    
    # Gather the log probabilities corresponding to the true classes
    gathered_probs = log_probs.gather(1, target.unsqueeze(1)).squeeze(1)
    if verbose: print("gathered_probs: ", gathered_probs.shape, gathered_probs)
    
    # Apply the mask
    masked_probs = gathered_probs * mask
    if verbose: print("masked_probs: ", masked_probs.shape, masked_probs)

    # Compute the average loss over unmasked entries
    # Use A.sum() to count the number of unmasked entries
    loss = -torch.sum(masked_probs) / torch.clamp(mask.sum(), min=1)
    
    return loss

### 5. NCP 3-step inference walkthrough

In [6]:
# but first, all the new hyperparameters & whatnot
gamma_0 = 0.5
t_max=2

vocabulary_dict_ext = vocabulary_dict
vocabulary_dict_ext[''] = 12

In [116]:
### FIRST PASS
verbose=False
batch_seq = [[' I'], ['Every']]

i=1
print(f"FIRST PASS: i={i}")

E, _, S_i = initialize(batch_seq, vocabulary_dict)
print("S_i: ", S_i.shape)
print("E: ", E.shape)
if verbose:
    print(S_i)
    print(E)

X_0 = F.embedding(S_i, E)
print("X_0: ", X_0.shape)
if verbose: print(X_0)

X_f = layers(X_0)
print("X_f: ", X_f.shape)
if verbose: print(X_f)

#Y = torch.matmul(X_f,E.T)
Y = X_f[:,-1,:]
print("Y: ", Y.shape)
if verbose: print(Y)

#Z = Y[:,-1,:]
Z = torch.matmul(Y,E.T)
print("Z: ", Z.shape)
if verbose: print(Z)

P = F.softmax(Z, dim=-1)
print("P: ", P.shape, torch.sum(P,dim=1))
if verbose: print(P)
    
P_max = torch.max(P, dim=1).values
print("P_max: ", P_max.shape, P_max)

s_iplus1 = torch.max(P, dim=1).indices
print("s_iplus1: ", s_iplus1.shape, s_iplus1, tensor_to_string(s_iplus1, vocabulary_dict))
if verbose: print(s_iplus1)
    
Gamma_i, A_i_bar, not_A_i_bar = initialize_conditionals(P_max)
print("Gamma_i: ", Gamma_i.shape, Gamma_i)
print("A_i_bar: ", A_i_bar.shape, A_i_bar)
print("not_A_i_bar: ", not_A_i_bar.shape, not_A_i_bar)

C_iplus1 = build_concepts_tensor(Y, A_i_bar)
print("C_iplus1: ", C_iplus1.shape, C_iplus1)

S_iplus1 = inference(A_i_bar, not_A_i_bar, s_iplus1, S_i)
print("S_iplus1: ", S_iplus1.shape, S_iplus1)
print(tensor_to_string(S_iplus1, vocabulary_dict_ext))




### SECOND PASS
i=2
print(f"SECOND PASS: i={i}")

# using outputs from previous iter
S_i, C_i = S_iplus1, C_iplus1
A_iminus1_bar, Gamma_iminus1 = A_i_bar, Gamma_i

# setting up our augmented E with an empty (zeros) vector for concept vectors
E_prime = torch.cat((E, torch.zeros((1,d))), dim=0)
print("E_prime: ", E_prime.shape)

# all of our token embedding vectors
T = F.embedding(S_i.type(torch.int32), E_prime)
print("T: ", T.shape, T)

# add correctly placed concept vectors and token embedding vectors to get our first resid
X_0 = C_i + T
print("X_0: ", X_0.shape)
if verbose: print(X_0)

X_f = layers(X_0)
print("X_f: ", X_f.shape)
if verbose: print(X_f)

#Y = torch.matmul(X_f,E.T)
Y = X_f[:,-1,:]
print("Y: ", Y.shape)
if verbose: print(Y)

#Z = Y[:,-1,:]
Z = torch.matmul(Y,E.T)
print("Z: ", Z.shape)
if verbose: print(Z)

P = F.softmax(Z, dim=-1)
print("P: ", P.shape, torch.sum(P,dim=1))
if verbose: print(P)
    
P_max = torch.max(P, dim=1).values
print("P_max: ", P_max.shape, P_max)

Gamma_i, A_i_bar, not_A_i_bar = iter_conditionals(P_max, A_iminus1_bar, Gamma_iminus1, t_max, gamma_0)
print("Gamma_i: ", Gamma_i.shape, Gamma_i)
print("A_i_bar: ", A_i_bar.shape, A_i_bar)
print("not_A_i_bar: ", not_A_i_bar.shape, not_A_i_bar)

C_iplus1 = build_concepts_tensor(Y, A_i_bar, C_i)
print("C_iplus1: ", C_iplus1.shape, C_iplus1)

S_iplus1 = inference(A_i_bar, not_A_i_bar, s_iplus1, S_i)
print("S_iplus1: ", S_iplus1.shape, S_iplus1)
print(tensor_to_string(S_iplus1, vocabulary_dict_ext))



### THIRD PASS
i=3
print(f"SECOND PASS: i={i}")

# using outputs from previous iter
S_i, C_i = S_iplus1, C_iplus1
A_iminus1_bar, Gamma_iminus1 = A_i_bar, Gamma_i

# setting up our augmented E with an empty (zeros) vector for concept vectors
E_prime = torch.cat((E, torch.zeros((1,d))), dim=0)
print("E_prime: ", E_prime.shape)

# all of our token embedding vectors
T = F.embedding(S_i.type(torch.int32), E_prime)
print("T: ", T.shape, T)

# add correctly placed concept vectors and token embedding vectors to get our first resid
X_0 = C_i + T
print("X_0: ", X_0.shape)
if verbose: print(X_0)

X_f = layers(X_0)
print("X_f: ", X_f.shape)
if verbose: print(X_f)

#Y = torch.matmul(X_f,E.T)
Y = X_f[:,-1,:]
print("Y: ", Y.shape)
if verbose: print(Y)

#Z = Y[:,-1,:]
Z = torch.matmul(Y,E.T)
print("Z: ", Z.shape)
if verbose: print(Z)

P = F.softmax(Z, dim=-1)
print("P: ", P.shape, torch.sum(P,dim=1))
if verbose: print(P)
    
P_max = torch.max(P, dim=1).values
print("P_max: ", P_max.shape, P_max)

Gamma_i, A_i_bar, not_A_i_bar = iter_conditionals(P_max, A_iminus1_bar, Gamma_iminus1, t_max, gamma_0)
print("Gamma_i: ", Gamma_i.shape, Gamma_i)
print("A_i_bar: ", A_i_bar.shape, A_i_bar)
print("not_A_i_bar: ", not_A_i_bar.shape, not_A_i_bar)

C_iplus1 = build_concepts_tensor(Y, A_i_bar, C_i)
print("C_iplus1: ", C_iplus1.shape, C_iplus1)

S_iplus1 = inference(A_i_bar, not_A_i_bar, s_iplus1, S_i)
print("S_iplus1: ", S_iplus1.shape, S_iplus1)
print(tensor_to_string(S_iplus1, vocabulary_dict_ext))


FIRST PASS: i=1
S_i:  torch.Size([2, 1])
E:  torch.Size([12, 4])
X_0:  torch.Size([2, 1, 4])
X_f:  torch.Size([2, 1, 4])
Y:  torch.Size([2, 4])
Z:  torch.Size([2, 12])
P:  torch.Size([2, 12]) tensor([1., 1.], grad_fn=<SumBackward1>)
P_max:  torch.Size([2]) tensor([0.2365, 0.6543], grad_fn=<MaxBackward0>)
s_iplus1:  torch.Size([2]) tensor([3, 7]) fore  has
Gamma_i:  torch.Size([2]) tensor([0.5000, 0.5000])
A_i_bar:  torch.Size([2]) tensor([1., 0.])
not_A_i_bar:  torch.Size([2]) tensor([0., 1.])
C_iplus1:  torch.Size([2, 2, 4]) tensor([[[ 0.0000,  0.0000,  0.0000,  0.0000],
         [ 1.1422, -1.5985,  0.0936,  0.3627]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000],
         [-0.0000,  0.0000, -0.0000,  0.0000]]], grad_fn=<CatBackward0>)
S_iplus1:  torch.Size([2, 2]) tensor([[ 0., 12.],
        [ 5.,  7.]])
[' I ', 'Every  has']
SECOND PASS: i=2
E_prime:  torch.Size([13, 4])
T:  torch.Size([2, 2, 4]) tensor([[[-1.3884, -0.5056,  0.8399,  1.0541],
         [ 0.0000,  0.0000,  0.0000,  0

### 6. NCP training loop

In [28]:
# since we no longer have a pre-set sequence length n, we now need
# to determine a maximum number of tokens t_max to add to prevent memory from overflowing
import numpy as np

gamma_0 = 0.5
C_max = 2
double_seq_list = [[' I', ' think', ' there', 'fore', ' I', ' am', '<endoftext>'],
               ['Every', ' cloud', ' has', ' a', ' silver', ' lining', '<endoftext>']]
n = len(double_seq_list[0])
b = len(double_seq_list)

t_max = np.floor(n*(np.sqrt(C_max)-1))
print("t_max: ", t_max)

vocabulary_dict_ext = vocabulary_dict
vocabulary_dict_ext[''] = 12

t_max:  2.0


In [48]:
verbose = False

for i in range(1,n+int(t_max)):
    print(f"********** ITERATION i={i} *************")
    
    # initialize
    if i == 1:
        E, S_n, S_i = initialize(double_seq_list, vocabulary_dict)
        print("S_n: ", S_n.shape, S_n)
        print("E: ", E.shape)
        if verbose:
            print(S_n)
            print(E)
            
        X_0 = F.embedding(S_i, E)
        print("X_0: ", X_0.shape)
        if verbose: print(X_0)
    else:
        S_i, C_i, not_A_i = S_iplus1, C_iplus1, not_A_iplus1
        A_iminus1_bar, Gamma_iminus1 = A_i_bar, Gamma_i
        
        # setting up our augmented E with an empty (zeros) vector for concept vectors
        E_prime = torch.cat((E, torch.zeros((1,d))), dim=0)
        print("E_prime: ", E_prime.shape)

        # all of our token embedding vectors
        T = F.embedding(S_i.type(torch.int32), E_prime)
        print("T: ", T.shape, T)

        # add correctly placed concept vectors and token embedding vectors to get our first resid
        X_0 = C_i + T
        print("X_0: ", X_0.shape)
        if verbose: print(X_0)
    

    X_f = layers(X_0)
    print("X_f: ", X_f.shape)
    if verbose: print(X_f)

    #Y = torch.matmul(X_f,E.T) #<-- regular GPT
    Y = X_f[:,-1,:]
    print("Y: ", Y.shape)
    if verbose: print(Y)

    #Z = Y[:,-1,:] #<-- regular GPT
    Z = torch.matmul(Y,E.T)
    print("Z: ", Z.shape)
    if verbose: print(Z)
    
    P = F.softmax(Z, dim=-1)
    print("P: ", P.shape, torch.sum(P,dim=1))
    if verbose: print(P)

    P_max = torch.max(P, dim=1).values
    print("P_max: ", P_max.shape, P_max)
    
    if i == 1:
        Gamma_i, A_i_bar, not_A_i_bar = initialize_conditionals(P_max)
        C_iplus1 = build_concepts_tensor(Y, A_i_bar)
    else:
        Gamma_i, A_i_bar, not_A_i_bar = iter_conditionals(P_max, A_iminus1_bar, Gamma_iminus1, t_max, gamma_0)
        C_iplus1 = build_concepts_tensor(Y, A_i_bar, C_i)
    print("Gamma_i: ", Gamma_i.shape, Gamma_i)
    print("A_i_bar: ", A_i_bar.shape, A_i_bar)
    print("not_A_i_bar: ", not_A_i_bar.shape, not_A_i_bar)
    print("C_iplus1: ", C_iplus1.shape, C_iplus1)
    
    # correct indices to be trained on
    if i == 1:
        s_iplus1 = S_n[:,i]
    else:
        print("not_A_i: ", not_A_i.shape, not_A_i)
        not_A_i_hat = not_A_i.sum(dim=1)
        print("not_A_i_hat: ", not_A_i_hat.shape, not_A_i_hat)
        if verbose:
            print("S_n: ", S_n.shape, S_n)
            print("S_n.size(0): ", S_n.size(0))
            print("torch.arange: ", torch.arange(S_n.size(0)))
        s_iplus1 = S_n[torch.arange(S_n.size(0)),not_A_i_hat.type(torch.int32)] #:
    print("s_iplus1: ", s_iplus1.shape, s_iplus1)
    
    # cross-entropy loss
    loss = custom_loss(Z, s_iplus1, not_A_i_bar)
    print("loss: ", loss.shape, loss)
    
    # so i guess i should've named this function something other than inference but whatever
    S_iplus1 = inference(A_i_bar, not_A_i_bar, s_iplus1, S_i)
    print("S_iplus1: ", S_iplus1.shape, S_iplus1)
    print(tensor_to_string(S_iplus1, vocabulary_dict_ext))
    
    if i == 1:
        not_A_iplus1 = torch.cat((torch.ones((b,i)), not_A_i_bar.unsqueeze(dim=1)), dim=1)
    else:
        not_A_iplus1 = torch.cat((not_A_i, not_A_i_bar.unsqueeze(dim=1)), dim=1)
    print("not_A_iplus1: ", not_A_iplus1.shape, not_A_iplus1)
    
### really i should add some logic to make it stop once each predicts <endoftext>
### and prevent sequences who've already predicted <endoftext> from keeping training
    

********** ITERATION i=1 *************
S_n:  torch.Size([2, 7]) tensor([[ 0,  1,  2,  3,  0,  4, 11],
        [ 5,  6,  7,  8,  9, 10, 11]])
E:  torch.Size([12, 4])
X_0:  torch.Size([2, 1, 4])
X_f:  torch.Size([2, 1, 4])
Y:  torch.Size([2, 4])
Z:  torch.Size([2, 12])
P:  torch.Size([2, 12]) tensor([1.0000, 1.0000], grad_fn=<SumBackward1>)
P_max:  torch.Size([2]) tensor([0.4952, 0.6295], grad_fn=<MaxBackward0>)
Gamma_i:  torch.Size([2]) tensor([0.5000, 0.5000])
A_i_bar:  torch.Size([2]) tensor([1., 0.])
not_A_i_bar:  torch.Size([2]) tensor([0., 1.])
C_iplus1:  torch.Size([2, 2, 4]) tensor([[[ 0.0000,  0.0000,  0.0000,  0.0000],
         [-0.0543, -0.2146,  1.5335, -1.2647]],

        [[ 0.0000,  0.0000,  0.0000,  0.0000],
         [-0.0000,  0.0000,  0.0000, -0.0000]]], grad_fn=<CatBackward0>)
s_iplus1:  torch.Size([2]) tensor([1, 6])
loss:  torch.Size([]) tensor(6.3587, grad_fn=<DivBackward0>)
S_iplus1:  torch.Size([2, 2]) tensor([[ 0., 12.],
        [ 5.,  6.]])
[' I ', 'Every  cloud'

IndexError: index 7 is out of bounds for dimension 1 with size 7